# Getting Started - Python SDK

---

## Setup

### Initialize HelixDB

In a terminal:

1. `cd python`
2. `helix init --path helixdb-cfg`

### Python Setup

### uv

1. In a terminal inside of directory `/python` run `uv venv`
2. Then `source .venv/bin/activate`
3. Then `uv sync`

### non-uv

1. In a terminal inside of directory `/python` run `python -m venv .venv`
2. Then `source .venv/bin/activate`
3. Then `pip install -r requirements.txt`

---

## Creating our Nodes

Inside of `helixdb-cfg/schema.hx` we can define the nodes we want:

```C++
N::Continent {
    name: String
}
```
```C++
N::Country {
    name: String,
    currency: String
}
```
```C++
N::City {
    name: String,
    description: String
}
```

## Creating our Edges between Nodes

```C++
E::Continent_to_Country {
    From: Continent,
    To: Country,
    Properties: {
    }
}

E::Country_to_City {
    From: Country,
    To: City,
    Properties: {
    }
}

E::Country_to_Capital {
    From: Country,
    To: City,
    Properties: {
    }
}
```

## Deploying HelixDB

1. cd into `helixdb-cfg`
2. run `helix deploy`

## Initializing Client using Python SDK

In [1]:
from pprint import pprint

In [ ]:
from helix.client import Client

client = Client(local=True, port=6969)

[HELIX] Helix instance found at 'http://127.0.0.1:6970'


---

## Creating our first query

Lets create our first query (`createContinent`) to insert in continent nodes, inside of `helixdb-cfg/queries.hx` add in the query below:

```C++
QUERY createContinent (name: String) =>
    continent <- AddN<Continent>({name: name})
    RETURN continent
```

### IMPORTANT: Redeploy HelixDB
After adding in a new QUERY you need to redeploy HelixDB
1. In a terminal, run `helix deploy -c <instance_id>`, if you don't know your instance_id you can do `helix status`

## Inserting in Data

### Continents

We will insert in two continents Europe and Asia

In [3]:
europe_node = client.query("createContinent",
                           {"name": "Europe"})

asia_node = client.query("createContinent",
                         {"name": "Asia"})

[HELIX] Querying 'http://127.0.0.1:6970/createContinent': 100%|██████████| 1/1 [00:00<00:00, 112.11it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createContinent': 100%|██████████| 1/1 [00:00<00:00, 162.10it/s]


In [4]:
europe_node

[{'continent': {'id': '1f0887ad-edc6-6ad0-b31c-010203040506',
   'label': 'Continent',
   'name': 'Europe'}}]

In [5]:
asia_node

[{'continent': {'id': '1f0887ad-eddf-6ecc-b31d-010203040506',
   'label': 'Continent',
   'name': 'Asia'}}]

### Inserting in Countries and Adding in an Edge between Continent and the Country

```C++
QUERY createCountry (continent_id: ID, name: String, currency: String) =>
    // Add a new country node with name and currency
    country <- AddN<Country>({name: name, currency: currency})
```
```C++
    // Find and link the country to the continent
    continent <- N<Continent>(continent_id)
    continent_country <- AddE<Continent_to_Country>()::From(continent)::To(country)
    RETURN country
```

*Dont forget to redeploy HelixDB!*

We're going to get the IDs for Europe and Asia so that we can use it in our `createCountry` QUERY

In [6]:
europe_id = europe_node[0]['continent']['id']
asia_id = asia_node[0]['continent']['id']
europe_id, asia_id

('1f0887ad-edc6-6ad0-b31c-010203040506',
 '1f0887ad-eddf-6ecc-b31d-010203040506')

In [7]:
united_kingdom_node = client.query("createCountry",
                                   {"continent_id": europe_id,
                                    "name": "United Kingdom",
                                    "currency": "GBP"})

germany_node = client.query("createCountry",
                            {"continent_id": europe_id,
                             "name": "Germany",
                             "currency": "Euro"})

china_node = client.query("createCountry",
                          {"continent_id": asia_id,
                           "name": "China",
                           "currency": "Yuan"})

[HELIX] Querying 'http://127.0.0.1:6970/createCountry': 100%|██████████| 1/1 [00:00<00:00, 181.03it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createCountry': 100%|██████████| 1/1 [00:00<00:00, 209.34it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createCountry': 100%|██████████| 1/1 [00:00<00:00, 196.35it/s]


In [8]:
united_kingdom_node

[{'country': {'currency': 'GBP',
   'name': 'United Kingdom',
   'id': '1f0887ad-ee26-6a70-b31e-010203040506',
   'label': 'Country'}}]

In [9]:
germany_node

[{'country': {'currency': 'Euro',
   'name': 'Germany',
   'label': 'Country',
   'id': '1f0887ad-ee36-666e-b320-010203040506'}}]

In [10]:
china_node

[{'country': {'currency': 'Yuan',
   'name': 'China',
   'label': 'Country',
   'id': '1f0887ad-ee44-626e-b322-010203040506'}}]

### Inserting in Cities and Adding in an Edge between Country and the City

```C++
QUERY createCity (country_id: ID, name: String, description: String) =>
    // Add a new city node with name and description
    city <- AddN<City>({name: name, description: description})

    // Find and link the city to the country
    country <- N<Country>(country_id)
    country_city <- AddE<Country_to_City>()::From(country)::To(city)
    RETURN city
```

*Dont forget to redeploy HelixDB!*

In [11]:
# Adding in London and Manchester to the United Kingdom
london_node = client.query("createCity",
                            {"country_id": united_kingdom_node[0]['country']['id'],
                            "name": "London",
                            "description": "Capital city known for its finance, culture, and history"})

manchester_node = client.query("createCity",
                            {"country_id": united_kingdom_node[0]['country']['id'],
                            "name": "Manchester",
                            "description": "Northern hub for music, sports, and industry"})

[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 175.69it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 190.04it/s]


In [12]:
# Adding in Berlin and Munich to Germany
berlin_node = client.query("createCity",
                           {"country_id": germany_node[0]['country']['id'],
                            "name": "Berlin",
                            "description": "Capital city known for history, culture, and politics"})

munich_node = client.query("createCity",
                           {"country_id": germany_node[0]['country']['id'],
                            "name": "Munich",
                            "description": "Bavarian capital known for Oktoberfest and tech innovation"})

[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 197.86it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 193.13it/s]


In [13]:
# Adding in Beijing and Shanghai to China
beijing_node = client.query("createCity",
                            {"country_id": china_node[0]['country']['id'],
                             "name": "Beijing",
                             "description": "Capital city with political, cultural, and historical importance"})

shanghai_node = client.query("createCity",
                             {"country_id": china_node[0]['country']['id'],
                              "name": "Shanghai",
                              "description": "Global financial hub and largest city by population"})

[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 196.97it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createCity': 100%|██████████| 1/1 [00:00<00:00, 193.77it/s]


### Adding in the Edge between a **Capital** City and a Country 

```C++
QUERY setCapital (country_id: ID, city_id: ID) =>
    // Find the country and city nodes
    country <- N<Country>(country_id)
    city <- N<City>(city_id)

    // Link the city to the country as capital
    country_capital <- AddE<Country_to_Capital>()::From(country)::To(city)
    RETURN country_capital
```

*Dont forget to redeploy HelixDB!*

In [14]:
# Adding in the London as the capital of the United Kingdom
pprint(client.query("setCapital",
             {"country_id": united_kingdom_node[0]['country']['id'],
              "city_id": london_node[0]['city']['id']}))

[HELIX] Querying 'http://127.0.0.1:6970/setCapital': 100%|██████████| 1/1 [00:00<00:00, 176.11it/s]

[{'country_capital': {'from_node': '1f0887ad-ee26-6a70-b31e-010203040506',
                      'id': '1f0887ad-eee4-6908-b330-010203040506',
                      'label': 'Country_to_Capital',
                      'to_node': '1f0887ad-ee75-67ec-b324-010203040506'}}]


In [15]:
# Adding in Berlin as the capital of Germany
client.query("setCapital",
             {"country_id": germany_node[0]['country']['id'],
              "city_id": berlin_node[0]['city']['id']})

[HELIX] Querying 'http://127.0.0.1:6970/setCapital': 100%|██████████| 1/1 [00:00<00:00, 195.96it/s]


[{'country_capital': {'id': '1f0887ad-eefb-6482-b331-010203040506',
   'label': 'Country_to_Capital',
   'from_node': '1f0887ad-ee36-666e-b320-010203040506',
   'to_node': '1f0887ad-ee9d-645e-b328-010203040506'}}]

In [16]:
client.query("setCapital",
             {"country_id": china_node[0]['country']['id'],
              "city_id": beijing_node[0]['city']['id']})

[HELIX] Querying 'http://127.0.0.1:6970/setCapital': 100%|██████████| 1/1 [00:00<00:00, 196.55it/s]


[{'country_capital': {'label': 'Country_to_Capital',
   'from_node': '1f0887ad-ee44-626e-b322-010203040506',
   'to_node': '1f0887ad-eec1-6214-b32c-010203040506',
   'id': '1f0887ad-ef11-6ae8-b332-010203040506'}}]

---

## Get queries

Now we have added in all Continents, Countries and their Cities. We can create `GET` queries in HelixDB

### Get all continents

```C++
QUERY getAllContinents () =>
    // Select all continent nodes
    continents <- N<Continent>
    RETURN continents
```

*Dont forget to redeploy HelixDB!*

In [17]:
pprint(client.query("getAllContinents", {}))

[HELIX] Querying 'http://127.0.0.1:6970/getAllContinents': 100%|██████████| 1/1 [00:00<00:00, 1326.05it/s]

[{'continents': [{'id': '1f0887ad-edc6-6ad0-b31c-010203040506',
                  'label': 'Continent',
                  'name': 'Europe'},
                 {'id': '1f0887ad-eddf-6ecc-b31d-010203040506',
                  'label': 'Continent',
                  'name': 'Asia'}]}]


### Get all countries

```C++
QUERY getAllContinents () =>
    // Select all continent nodes
    continents <- N<Continent>
    RETURN continents
```
*Dont forget to redeploy HelixDB!*


In [18]:
pprint(client.query("getAllCountries", {}))

[HELIX] Querying 'http://127.0.0.1:6970/getAllCountries': 100%|██████████| 1/1 [00:00<00:00, 1475.31it/s]

[{'countries': [{'currency': 'GBP',
                 'id': '1f0887ad-ee26-6a70-b31e-010203040506',
                 'label': 'Country',
                 'name': 'United Kingdom'},
                {'currency': 'Euro',
                 'id': '1f0887ad-ee36-666e-b320-010203040506',
                 'label': 'Country',
                 'name': 'Germany'},
                {'currency': 'Yuan',
                 'id': '1f0887ad-ee44-626e-b322-010203040506',
                 'label': 'Country',
                 'name': 'China'}]}]


### Get all countries by continent ID

```C++
QUERY getAllCountriesByContinentID (continent_id: ID) =>
    // Select continent node with given ID
    continent <- N<Continent>(continent_id)

    // Select all countries linked to the continent
    countries <- continent::Out<Continent_to_Country>
    RETURN countries
```

*Dont forget to redeploy HelixDB!*

In [19]:
pprint(client.query("getAllCountriesByContinentID", {"continent_id": europe_id}))

[HELIX] Querying 'http://127.0.0.1:6970/getAllCountriesByContinentID': 100%|██████████| 1/1 [00:00<00:00, 1120.87it/s]

[{'countries': [{'currency': 'GBP',
                 'id': '1f0887ad-ee26-6a70-b31e-010203040506',
                 'label': 'Country',
                 'name': 'United Kingdom'},
                {'currency': 'Euro',
                 'id': '1f0887ad-ee36-666e-b320-010203040506',
                 'label': 'Country',
                 'name': 'Germany'}]}]


```C++
QUERY getAllCitiesByCountryID (country_id: ID) =>
    // Select country node with given ID
    country <- N<Country>(country_id)

    // Select all cities linked to the country
    cities <- country::Out<Country_to_City>
    RETURN cities
```
*Dont forget to redeploy HelixDB!*

In [20]:
pprint(client.query("getAllCitiesByCountryID", {"country_id": united_kingdom_node[0]['country']['id']}))

[HELIX] Querying 'http://127.0.0.1:6970/getAllCitiesByCountryID': 100%|██████████| 1/1 [00:00<00:00, 1264.49it/s]

[{'cities': [{'description': 'Capital city known for its finance, culture, and '
                             'history',
              'id': '1f0887ad-ee75-67ec-b324-010203040506',
              'label': 'City',
              'name': 'London'},
             {'description': 'Northern hub for music, sports, and industry',
              'id': '1f0887ad-ee86-6010-b326-010203040506',
              'label': 'City',
              'name': 'Manchester'}]}]


### Getting Capital City of a Country

```C++
QUERY getCapital (country_id: ID) =>
    // Select country node with given ID
    country <- N<Country>(country_id)

    // Select capital node linked to the country
    capital <- country::Out<Country_to_Capital>
    RETURN capital
```

*Dont forget to redeploy HelixDB!*

In [21]:
pprint(client.query("getCapital", {"country_id": united_kingdom_node[0]['country']['id']}))

[HELIX] Querying 'http://127.0.0.1:6970/getCapital': 100%|██████████| 1/1 [00:00<00:00, 1203.53it/s]

[{'capital': [{'description': 'Capital city known for its finance, culture, '
                              'and history',
               'id': '1f0887ad-ee75-67ec-b324-010203040506',
               'label': 'City',
               'name': 'London'}]}]


## Adding and Searching Embeddings

For embeddings we can add in a vector node into our `schema.hx`.

```C++
V::Description_Embedding {
    embedding: [F64]
}

E::Description_Embedding_to_City {
    From: Description_Embedding,
    To: City,
    Properties: {
    }
}
```

We will be embedding the description of the city, and connect that vector to the city node. 

*Dont forget to redeploy HelixDB!*

And in our `query.hx`, you can add in:

```C++
QUERY createDescriptionEmbedding (city_id: ID, embedding: [F64]) =>
    description_embedding <- AddV<Description_Embedding>(embedding)
    city <- N<City>(city_id)
    city_description_embedding <- AddE<Description_Embedding_to_City>()::From(city)::To(description_embedding)
    RETURN description_embedding
```

```C++
QUERY searchDescriptions (vector: [F64], k: I64) =>
    descriptions <- SearchV<Description_Embedding>(vector, k)
    cities <- descriptions::In<Description_Embedding_to_City>
    RETURN cities
```

*Dont forget to redeploy HelixDB!*

### Adding in description embeddings for each city

In [22]:
client.query("createDescriptionEmbedding", 
             {"city_id": london_node[0]['city']['id'],
              "embedding": [0.1, 0.2, 0.3]})

client.query("createDescriptionEmbedding",
             {"city_id": manchester_node[0]['city']['id'],
              "embedding": [0.2, 0.3, 0.4]})

client.query("createDescriptionEmbedding",
             {"city_id": berlin_node[0]['city']['id'],
              "embedding": [0.3, 0.4, 0.5]})

client.query("createDescriptionEmbedding",
             {"city_id": munich_node[0]['city']['id'],
              "embedding": [0.4, 0.5, 0.6]})

client.query("createDescriptionEmbedding",
             {"city_id": beijing_node[0]['city']['id'],
              "embedding": [0.5, 0.6, 0.7]})

client.query("createDescriptionEmbedding",
             {"city_id": shanghai_node[0]['city']['id'],
              "embedding": [0.6, 0.7, 0.8]})

[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 183.59it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 195.58it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 77.34it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 191.53it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 194.28it/s]
[HELIX] Querying 'http://127.0.0.1:6970/createDescriptionEmbedding': 100%|██████████| 1/1 [00:00<00:00, 173.43it/s]


[{'description_embedding': {'data': [0.6, 0.7, 0.8],
   'label': 'vector',
   'id': '1f0887ad-efc2-650a-b33d-010203040506',
   'score': 2.0}}]

### Searching vector nodes by similarity search

In [24]:
pprint(client.query("searchDescriptions", {"vector": [0.6, 0.7, 0.8], "k": 1}))

[HELIX] Querying 'http://127.0.0.1:6970/searchDescriptions': 100%|██████████| 1/1 [00:00<00:00, 1007.04it/s]

[{'cities': [{'description': 'Global financial hub and largest city by '
                             'population',
              'id': '1f0887ad-eecf-6120-b32e-010203040506',
              'label': 'City',
              'name': 'Shanghai'}]}]
